In [1]:
#importing packages needed to complete project 2
import requests
import pandas as pd
import json
import time
from bs4 import BeautifulSoup

key = "1nOoUuH_Q1Fve7aRtOI9jjxW9ko8MbHB"
    

In [2]:

#scraping wiki table of Nasdaq-100
url = "https://en.wikipedia.org/wiki/Nasdaq-100"
page= requests.get(url,verify=True)
soup = BeautifulSoup(page.content,'html.parser')
#scraping all table tags with id = constituents into SP variable
table = soup.find('table', id="constituents")

#finding all of the rows inside the variable table
rows= table.findAll('tr')
    
#creating matrix for wiki table (df1)
Company=[]
Ticker=[]
Sector=[]
Sub_industry=[]

#creating a loop to append data into respective dataframe matrixes
for row in rows:
    cells = row.findAll('td')
    th = row.findAll('th')
    if len(cells) ==4:
        Company.append(cells[0].find(text=True)) 
        Ticker.append(cells[1].find(text=True))
        Sector.append(cells[2].find(text=True))
        Sub_industry.append(cells[3].find(text=True).rstrip("\n"))
    
#creating dataframe for wikitable (nasdaq 100)
df1= pd.DataFrame()
df1["Company"]= Company
df1["Ticker"]= Ticker
df1["Sector"]= Sector
df1["Sub Industry"] = Sub_industry
df1.set_index("Company", inplace= True)



,Ticker,Sector,Sub Industry
Company,,,
Activision Blizzard,ATVI,Communication Services,Interactive Home Entertainment
Adobe Inc.,ADBE,Information Technology,Application Software
Advanced Micro Devices,AMD,Information Technology,Semiconductors
Alexion Pharmaceuticals,ALXN,Health Care,Pharmaceuticals
Align Technology,ALGN,Health Care,Health Care Supplies
...,...,...,...
"Walgreen Boots Alliance, Inc.",WBA,Consumer Staples,Drug Retail
"Workday, Inc.",WDAY,Information Technology,Application Software
Xcel Energy Inc.,XEL,Utilities,Multi-Utilities


In [3]:
#creating a matrix for second part of assignment
Open=[]
close=[]
low=[]
high=[]
volume=[]
symbol=[]

#creating a loop to request information from API.
for stocks in Ticker:     #using tickers from scraped table as an API query 
        url = "https://api.polygon.io/v1/open-close/"+stocks+"/2020-10-14?unadjusted=true&apiKey="+ key
        #requesting information from api
        r = requests.get(url) 
        #response in JSON
        data=r.json()
        #API only restricts requests to 5 per 60s
        time.sleep(12) 
        #printing formatted dictionary
        print(json.dumps(data, sort_keys=True, indent=10))
        
        #Appending information from API into matrix
        Open.append(data["open"])
        close.append(data["close"])
        low.append(data["low"])
        high.append(data["high"])
        volume.append(data["volume"])
        symbol.append(data["symbol"])

{
          "afterHours": 81.86,
          "close": 81.9,
          "from": "2020-10-14",
          "high": 82.96,
          "low": 81.0768,
          "open": 82.51,
          "preMarket": 81.97,
          "status": "OK",
          "symbol": "ATVI",
          "volume": 5516890
}
{
          "afterHours": 504,
          "close": 506.31,
          "from": "2020-10-14",
          "high": 516.95,
          "low": 501.6,
          "open": 514.34,
          "preMarket": 515.99,
          "status": "OK",
          "symbol": "ADBE",
          "volume": 2033159
}
{
          "afterHours": 83.97,
          "close": 84.21,
          "from": "2020-10-14",
          "high": 85.96,
          "low": 83.46,
          "open": 85.86,
          "preMarket": 85.98,
          "status": "OK",
          "symbol": "AMD",
          "volume": 40445635
}
{
          "afterHours": 122.52,
          "close": 122.52,
          "from": "2020-10-14",
          "high": 124.42,
          "low": 122.21,
          "open"

In [17]:
#creating second dataframe 
df2= pd.DataFrame()
df2["Ticker"]= symbol
df2["Opening Price"]=Open
df2["Closing Price"]= close
df2["Low of the Day"]= low
df2['High of the Day']= high 
df2['Shares Traded Today']= volume
#rounding to 2 decimal places
df2= round(df2,2)
#Printing second dataframe information
df2

,Ticker,Opening Price,Closing Price,Low of the Day,High of the Day,Shares Traded Today
0,ATVI,82.51,81.90,81.08,82.96,5516890
1,ADBE,514.34,506.31,501.60,516.95,2033159
2,AMD,85.86,84.21,83.46,85.96,40445635
3,ALXN,124.11,122.52,122.21,124.42,1883948
4,ALGN,333.53,333.10,328.39,337.20,555454
...,...,...,...,...,...,...
97,WBA,36.32,35.91,35.65,36.81,8381970
98,WDAY,236.60,230.54,227.55,236.60,1209071
99,XEL,73.35,72.35,72.06,73.35,1722727
100,XLNX,117.36,117.75,116.50,118.75,2473543


In [18]:
#creating third dataframe. Merging df1 and df2 through 'Ticker' 
combined_df= pd.merge(df1, df2, how= 'outer', on="Ticker")
#statistical data on the merged dataframe
combined_df.describe()

,Opening Price,Closing Price,Low of the Day,High of the Day,Shares Traded Today
count,102.000000,102.000000,102.000000,102.000000,1.020000e+02
mean,277.862941,273.892647,271.431275,280.030686,6.604456e+06
std,443.671603,434.348775,430.958705,446.174564,1.638095e+07
min,5.890000,5.840000,5.810000,5.920000,3.143300e+05
25%,82.925000,82.090000,81.352500,83.440000,1.109750e+06
50%,129.730000,130.660000,128.595000,133.140000,2.120270e+06
75%,323.395000,319.845000,316.180000,326.147500,5.827694e+06
max,3447.000000,3363.710000,3340.000000,3464.880000,1.510572e+08


In [19]:
#converting df3 into csv file
combined_df.to_csv(r'C:\Users\willi\Documents\Cis3120\WilliamZhuProj2_CSV', index = False)